### Pre-processing of ERA5 climate data


In [1]:
import sys
import os
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray
import rasterio
import matplotlib.pyplot as plt
os.chdir('/home/cshatto/Projects/effis-era5-europe')

# import ee
import scripts.fwi_functions as fwi_functions

-----
Load in external geometries for clipping to EEA-38 later on.

In [ ]:
# load Europe shapefile
europe_outline = gpd.read_file('data/processed/europe/Europe_outline.gpkg')
europe_eea38 = gpd.read_file('data/processed/europe/Europe_EEA38.gpkg')

# Convert the shapely geometry to a GeoJSON-like dict using __geo_interface__
geojson_geom = europe_outline.__geo_interface__



And open previously pre-processed datasets (skip future steps as needed).

In [ ]:
# era5 = rioxarray.open_rasterio('data/processed/era5-land/era5_land_monthly_fwi+vpd.nc')
# era5['FWI']

-----
Load in raw data. Open using rioxarray.

In [14]:
# era5_path_0 = "data/raw/era5-land/a4e067cd0162f1e6d9b2c3bbc7dc7ca6.nc"
era5_path_1 = "data/raw/e-obs/1099a23fb2eef99b26cd416ccfa8c666/fg_ens_mean_0.1deg_reg_2011-2024_v30.0e.nc"
era5_path_2 = "data/raw/e-obs/1099a23fb2eef99b26cd416ccfa8c666/rr_ens_mean_0.1deg_reg_2011-2024_v30.0e.nc"
era5_path_3 = "data/raw/e-obs/1099a23fb2eef99b26cd416ccfa8c666/tx_ens_mean_0.1deg_reg_2011-2024_v30.0e.nc"
era5_path_4 = "data/raw/e-obs/7855bc67929607b2062ffe819007ff59/fg_ens_mean_0.1deg_reg_1995-2010_v30.0e.nc"
era5_path_5 = "data/raw/e-obs/7855bc67929607b2062ffe819007ff59/rr_ens_mean_0.1deg_reg_1995-2010_v30.0e.nc"
era5_path_6 = "data/raw/e-obs/7855bc67929607b2062ffe819007ff59/tx_ens_mean_0.1deg_reg_1995-2010_v30.0e.nc"

# era5_0 = rioxarray.open_rasterio(era5_path_0)
era5_1 = rioxarray.open_rasterio(era5_path_1)
era5_2 = rioxarray.open_rasterio(era5_path_2)
era5_3 = rioxarray.open_rasterio(era5_path_3)
era5_4 = rioxarray.open_rasterio(era5_path_4)
era5_5 = rioxarray.open_rasterio(era5_path_5)
era5_6 = rioxarray.open_rasterio(era5_path_6)

# era5_0# 

In [7]:
era5_1

<xarray.DataArray 'fg' (time: 162, y: 465, x: 705)> Size: 212MB
[53107650 values with dtype=float32]
Coordinates:
  * x            (x) float64 6kB -24.95 -24.85 -24.75 ... 45.25 45.35 45.45
  * y            (y) float64 4kB 71.45 71.35 71.25 71.15 ... 25.25 25.15 25.05
  * time         (time) object 1kB 2011-01-31 00:00:00 ... 2024-06-30 00:00:00
    spatial_ref  int64 8B 0
Attributes: (12/26)
    latitude#axis:            Y
    latitude#long_name:       Latitude values
    latitude#standard_name:   latitude
    latitude#units:           degrees_north
    latitude#_FillValue:      nan
    longitude#axis:           X
    ...                       ...
    standard_name:            wind_speed
    long_name:                Ensemble mean wind speed
    units:                    ('m/s', 'm/s', 'm/s', 'm/s', 'm/s', 'm/s', 'm/s...
    _FillValue:               nan
    scale_factor:             1.0
    add_offset:               0.0

In [5]:
files = [era5_path_1, era5_path_2, era5_path_3,
         era5_path_4, era5_path_5, era5_path_6]

# Process each file separately
for file in files:
    # Load the individual NetCDF file
    ds = xr.open_dataset(file)
    
    # Compute monthly averages
    monthly_avg = ds.resample(time='1ME').mean()
    
    # Save the monthly average to a new file
    output_file = file.replace('.nc', '_monthly_avg.nc')
    monthly_avg.to_netcdf(output_file)
    print(f"Saved monthly average for {file} to {output_file}")
    
    # Close the dataset to free memory
    ds.close()
    monthly_avg.close()


Saved monthly average for data/raw/e-obs/1099a23fb2eef99b26cd416ccfa8c666/fg_ens_mean_0.1deg_reg_2011-2024_v30.0e.nc to data/raw/e-obs/1099a23fb2eef99b26cd416ccfa8c666/fg_ens_mean_0.1deg_reg_2011-2024_v30.0e_monthly_avg.nc
Saved monthly average for data/raw/e-obs/1099a23fb2eef99b26cd416ccfa8c666/rr_ens_mean_0.1deg_reg_2011-2024_v30.0e.nc to data/raw/e-obs/1099a23fb2eef99b26cd416ccfa8c666/rr_ens_mean_0.1deg_reg_2011-2024_v30.0e_monthly_avg.nc
Saved monthly average for data/raw/e-obs/1099a23fb2eef99b26cd416ccfa8c666/tx_ens_mean_0.1deg_reg_2011-2024_v30.0e.nc to data/raw/e-obs/1099a23fb2eef99b26cd416ccfa8c666/tx_ens_mean_0.1deg_reg_2011-2024_v30.0e_monthly_avg.nc
Saved monthly average for data/raw/e-obs/7855bc67929607b2062ffe819007ff59/fg_ens_mean_0.1deg_reg_1995-2010_v30.0e.nc to data/raw/e-obs/7855bc67929607b2062ffe819007ff59/fg_ens_mean_0.1deg_reg_1995-2010_v30.0e_monthly_avg.nc
Saved monthly average for data/raw/e-obs/7855bc67929607b2062ffe819007ff59/rr_ens_mean_0.1deg_reg_1995-2010_v

In [15]:
import xarray as xr
import pandas as pd

# Your file paths (update these to match your actual paths)
files = [era5_path_1, era5_path_2, era5_path_3, era5_path_4, era5_path_5, era5_path_6]

# Dictionary to group files by prefix
prefix_groups = {'fg_': [], 'rr_': [], 'tx_': []}

# Group files by prefix (assuming monthly averaged files exist)
for file in files:
    monthly_file = file.replace('.nc', '_monthly_avg.nc')
    if 'fg_' in monthly_file:
        prefix_groups['fg_'].append(monthly_file)
    elif 'rr_' in monthly_file:
        prefix_groups['rr_'].append(monthly_file)
    elif 'tx_' in monthly_file:
        prefix_groups['tx_'].append(monthly_file)

# Load, merge, and filter each prefix group
merged_datasets = {}
for prefix, file_list in prefix_groups.items():
    if len(file_list) != 2:
        print(f"Warning: Expected 2 files for {prefix}, found {len(file_list)}")
        continue
    
    # Load the two files
    ds1 = xr.open_dataset(file_list[0])
    ds2 = xr.open_dataset(file_list[1])
    
    # Merge along time dimension
    combined = xr.concat([ds1, ds2], dim='time')
    
    # Sort by time to ensure chronological order
    combined = combined.sortby('time')
    
    # Filter to 2000-2024
    combined = combined.sel(time=slice('2000-01-01', '2024-12-31'))
    
    # Store in dictionary
    merged_datasets[prefix] = combined
    print(f"Merged {prefix} dataset: {combined.time.values[0]} to {combined.time.values[-1]}")
    
    # Close individual datasets
    ds1.close()
    ds2.close()

# Example: Access merged datasets
fg_data = merged_datasets['fg_']
rr_data = merged_datasets['rr_']
tx_data = merged_datasets['tx_']

Merged fg_ dataset: 2000-01-31T00:00:00.000000000 to 2024-06-30T00:00:00.000000000
Merged rr_ dataset: 2000-01-31T00:00:00.000000000 to 2024-06-30T00:00:00.000000000
Merged tx_ dataset: 2000-01-31T00:00:00.000000000 to 2024-06-30T00:00:00.000000000


In [17]:
tx_data

<xarray.Dataset> Size: 386MB
Dimensions:    (time: 294, latitude: 465, longitude: 705)
Coordinates:
  * latitude   (latitude) float64 4kB 25.05 25.15 25.25 ... 71.25 71.35 71.45
  * longitude  (longitude) float64 6kB -24.95 -24.85 -24.75 ... 45.35 45.45
  * time       (time) datetime64[ns] 2kB 2000-01-31 2000-02-29 ... 2024-06-30
Data variables:
    tx         (time, latitude, longitude) float32 386MB nan nan nan ... nan nan
Attributes:
    E-OBS_version:  30.0e
    Conventions:    CF-1.4
    References:     http://surfobs.climate.copernicus.eu/dataaccess/access_eo...
    history:        Fri Aug 30 11:42:54 2024: ncks --no-abc -d time,22280,272...
    NCO:            netCDF Operators version 5.1.8 (Homepage = http://nco.sf....

In [ ]:
plt.figure(figsize=(10, 6))
era5_1_M[1].plot(label=f'Monthly Average {variable}')
plt.title(f'Monthly Average {variable} (Spatially Averaged)')
plt.xlabel('Time')
plt.ylabel(f'{variable} (units: {monthly_avg[variable].units})')
plt.grid(True)
plt.legend()
plt.show()

-----
### Convert values to required units
- Temperature Kelvin -> Celcius
- Precipitation m -> mm
- Relative Humidity %
- Wind Speed m/s -> km/h

Temperature & RH (Calculate RH first)

In [ ]:
# Convert temperatures from Kelvin to Celsius.
t2m_C = era5_0['t2m'] - 273.15
d2m_C = era5_0['d2m'] - 273.15

# Compute saturation vapor pressure (e_s) and actual vapor pressure (e_a) using the Tetens formula.
# Formula: e = 6.112 * exp((17.67 * T) / (T + 243.5))  where T is in Celsius, result in hPa.
e_s = 6.112 * np.exp((17.67 * t2m_C) / (t2m_C + 243.5))
e_a = 6.112 * np.exp((17.67 * d2m_C) / (d2m_C + 243.5))

vpd_tetens = e_s - e_a

# Add VPD to the dataset
era5_0['vpd_tetens'] = vpd_tetens

# Calculate relative humidity (RH) in percentage.
RH = 100 * (e_a / e_s)

# Add the RH as a new data variable to the combined dataset.
era5_0 = era5_0.assign(RH=RH)

In [ ]:
era5_0 = era5_0.assign(t2m_C=t2m_C)
era5_0 = era5_0.assign(d2m_C=d2m_C)

In [ ]:
# Define the August-Roche-Magnus function
def calc_vapor_pressure(temp_c):
    """Calculate vapor pressure (kPa) using August-Roche-Magnus formula."""
    return 0.61094 * np.exp((17.625 * temp_c) / (temp_c + 243.04))

# Calculate saturation vapor pressure (e_s) from t2m_C
e_s = calc_vapor_pressure(era5_0['t2m_C'])

# Calculate actual vapor pressure (e_a) from d2m_C
e_a = calc_vapor_pressure(era5_0['d2m_C'])

# Calculate VPD
vpd = e_s - e_a

# Add VPD to the dataset
era5_0['vpd_ARM'] = vpd

Wind speed

In [ ]:
# Calculate wind speed using the Pythagorean theorem.
# windspeed = sqrt(u10^2 + v10^2)
windspeed_m_s = np.sqrt(era5_0['u10']**2 + era5_0['v10']**2)
windspeed_km_h = windspeed_m_s * 3.6
# Add windspeed as a new data variable to the combined dataset.
era5_0['windspeed_km_h'] = windspeed_km_h 

# era5_0 = era5_0.assign(windspeed=windspeed_km_h)

Precipitation

In [ ]:
# Convert precipitation from meters to millimeters
era5_0['tp_mm'] = era5_0['tp'] * 1000

Save to netCDF before processing FWI + VPD.
(Optional, processing so far not so heavy)

In [ ]:
import netCDF4
# Save era5_0 to a new NetCDF file
output_path = "data/processed/era5-land/era5_land_monthly.nc"
era5_0.to_netcdf(output_path, mode='w', format='NETCDF4', engine='netcdf4')

In [ ]:
era5_0 = rioxarray.open_rasterio("data/processed/era5-land/era5_land_monthly.nc")


### Calculate FWI from van Wagner 1987

In [ ]:
### FWI Calculation #1
"""
Compute the Canadian Forest Fire Weather Index (FWI) from daily meteorological inputs.
The formulas are based on Van Wagner (1987) and subsequent refinements.
Note: This implementation assumes that:
    - Temperature is provided in °C.
    - Relative Humidity is in %.
    - Wind speed is provided in km/h.
    - Precipitation is in mm.
The system is recursive, so previous day values for FFMC, DMC, and DC must be supplied.
"""

import numpy as np 
import xarray as xr 

def calc_ffmc(T, RH, wind, rain, ffmc_yesterday):
    """
    Calculate the Fine Fuel Moisture Code (FFMC).
    
    Parameters:
      T : xarray.DataArray or np.array
          Daily air temperature in °C.
      RH : xarray.DataArray or np.array
          Daily relative humidity in %.
      wind : xarray.DataArray or np.array
          Daily wind speed in km/h.
      rain : xarray.DataArray or np.array
          Daily precipitation in mm.
      ffmc_yesterday : xarray.DataArray or np.array
          Yesterday's FFMC (unitless, typically initialized around 85).
          
    Returns:
      ffmc : xarray.DataArray or np.array
          Today's FFMC.
    """
    # Convert previous FFMC to moisture content (m_o)
    m_o = 147.2 * (101.0 - ffmc_yesterday) / (59.5 + ffmc_yesterday)
    
    # Rainfall effect: only effective if rain > 0.5 mm
    # Effective rainfall (rf)
    rf = xr.where(rain > 0.5, rain - 0.5, 0.0)
    
    # Update moisture content after rain (m_r)
    # The following equations adjust m_o by the rain effect.
    # There are two regimes depending on whether m_o is below or above 150.
    m_r = xr.where(
        rf > 0,
        xr.where(m_o <= 150.0,
                 m_o + 42.5 * rf * np.exp(-100.0 / (251.0 - m_o)) * (1 - np.exp(-6.93 / rf)),
                 m_o + 42.5 * rf * np.exp(-100.0 / (251.0 - m_o)) * (1 - np.exp(-6.93 / rf)) + 0.0015 * (m_o - 150.0)**2 * np.sqrt(rf)
                ),
        m_o
    )
    # Cap moisture content at 250
    m_r = xr.where(m_r > 250.0, 250.0, m_r)
    
    # Equilibrium moisture content for drying (E_d) and wetting (E_w)
    E_d = 0.942 * (RH**0.679) + 11.0 * np.exp((RH - 100.0) / 10.0) + 0.18 * (21.1 - T) * (1 - np.exp(-0.115 * RH))
    E_w = 0.618 * (RH**0.753) + 10.0 * np.exp((RH - 100.0) / 10.0) + 0.18 * (21.1 - T) * (1 - np.exp(-0.115 * RH))
    
    # Drying or wetting phase: if m_r > E_d, drying; if m_r < E_w, wetting; otherwise, no change.
    # Calculate log drying and wetting rates:
    k_d = (0.424 * (1 - (RH / 100.0)**1.7) + 0.0694 * np.sqrt(wind) * (1 - (RH / 100.0)**8)) * 0.581 * np.exp(0.0365 * T)
    k_w = (0.424 * (1 - ((100.0 - RH) / 100.0)**1.7) + 0.0694 * np.sqrt(wind) * (1 - ((100.0 - RH) / 100.0)**8)) * 0.581 * np.exp(0.0365 * T)
    
    m = xr.where(m_r > E_d, E_d + (m_r - E_d) / (10**k_d),
         xr.where(m_r < E_w, E_w - (E_w - m_r) / (10**k_w),
                  m_r))
    
    # Convert moisture content back to FFMC
    ffmc = (59.5 * (250.0 - m)) / (147.2 + m)
    ffmc = xr.where(ffmc > 101.0, 101.0, ffmc)
    ffmc = xr.where(ffmc < 0.0, 0.0, ffmc)
    return ffmc

def calc_dmc(T, RH, rain, dmc_yesterday):
    """
    Calculate the Duff Moisture Code (DMC).
    
    Parameters:
      T : xarray.DataArray or np.array
          Daily air temperature in °C.
      RH : xarray.DataArray or np.array
          Daily relative humidity in %.
      rain : xarray.DataArray or np.array
          Daily precipitation in mm.
      dmc_yesterday : xarray.DataArray or np.array
          Yesterday's DMC.
          
    Returns:
      dmc : xarray.DataArray or np.array
          Today's DMC.
    """
    # Effective rainfall for DMC (in mm); threshold of 1.5 mm.
    rf = xr.where(rain > 1.5, rain, 0.0)
    # Rain effect: update DMC due to rain (Re)
    # Re = 0.92 * rf - 1.27 if rf > 1.5 else 0.0  # Note: vectorizse below
    Re = xr.where(rain > 1.5, 0.92 * rain - 1.27, 0.0)
    
    # Calculate adjustment term for DMC (based on temperature and RH)
    # Following Van Wagner (1987), use:
    # K = 1.894 * (T + 1.1) * (100 - RH) * 0.0001
    K = 1.894 * (T + 1.1) * (100 - RH) * 1e-4
    # DMC increases by rain-adjusted term plus the drying term K.
    dmc = np.maximum(dmc_yesterday + Re + K, 0)
    return dmc

def calc_dc(T, rain, dc_yesterday):
    """
    Calculate the Drought Code (DC).
    
    Parameters:
      T : xarray.DataArray or np.array
          Daily air temperature in °C.
      rain : xarray.DataArray or np.array
          Daily precipitation in mm.
      dc_yesterday : xarray.DataArray or np.array
          Yesterday's DC.
          
    Returns:
      dc : xarray.DataArray or np.array
          Today's DC.
    """
    # Effective rainfall for DC: only if rain > 2.8 mm.
    rf = xr.where(rain > 2.8, rain, 0.0)
    Rd = xr.where(rain > 2.8, 0.83 * rain - 1.27, 0.0)
    # Potential evapotranspiration term: V = 0.36*(T+2.8) if T>= -2.8, else 0.
    V = xr.where(T >= -2.8, 0.36 * (T + 2.8), 0.0)
    dc = dc_yesterday + 0.5 * V
    # Adjust for rain: if rain > 2.8, update DC based on Rd.
    # Following Van Wagner (1987): Qo = 800 * exp(-dc_yesterday/400)
    Qo = 800 * np.exp(-dc_yesterday / 400.0)
    Qr = Qo + 3.937 * Rd
    Dr = 400 * np.log(800.0 / Qr)
    dc = xr.where(rain > 2.8, Dr, dc)
    dc = np.maximum(dc, 0)
    return dc

def calc_isi(ffmc, wind):
    """
    Calculate the Initial Spread Index (ISI) based on FFMC and wind.
    
    Parameters:
      ffmc : xarray.DataArray or np.array
          Today's FFMC.
      wind : xarray.DataArray or np.array
          Wind speed in km/h.
          
    Returns:
      isi : xarray.DataArray or np.array
          Initial Spread Index.
    """
    # Convert FFMC to fine fuel moisture content (m)
    m = (59.5 * 250 - 147.2 * ffmc) / (ffmc + 59.5)
    # Fine fuel moisture function:
    f_F = 91.9 * np.exp(-0.1386 * m) * (1 + (m**5.31) / 4.93e7)
    # Wind function: f(W) = exp(0.05039 * wind)
    f_W = np.exp(0.05039 * wind)
    isi = 0.208 * f_W * f_F
    return isi

def calc_bui(dmc, dc):
    """
    Calculate the Buildup Index (BUI) from DMC and DC.
    
    Parameters:
      dmc : xarray.DataArray or np.array
          Duff Moisture Code.
      dc : xarray.DataArray or np.array
          Drought Code.
          
    Returns:
      bui : xarray.DataArray or np.array
          Buildup Index.
    """
    bui = xr.zeros_like(dmc)
    cond = dmc <= 0.4 * dc
    bui = xr.where(cond, (0.8 * dmc * dc) / (dmc + 0.4 * dc),
                     dmc - (1 - 0.8 * dc / (dmc + 0.4 * dc)) * (0.92 + 0.0114 * dmc))
    bui = np.maximum(bui, 0)
    return bui

def calc_fwi(isi, bui):
    """
    Calculate the final Fire Weather Index (FWI) from ISI and BUI.
    
    Parameters:
      isi : xarray.DataArray or np.array
          Initial Spread Index.
      bui : xarray.DataArray or np.array
          Buildup Index.
          
    Returns:
      fwi : xarray.DataArray or np.array
          Fire Weather Index.
    """
    # First, compute an intermediate fire intensity index, B:
    # f(D) is defined piecewise:
    f_D = xr.where(bui <= 80, 0.626 * (bui ** 0.809) + 2,
                   1000.0 / (25.0 + 108.64 * np.exp(-0.023 * bui)))
    B = 0.1 * isi * f_D
    # Then, final FWI:
    fwi = xr.where(B > 1, np.exp(2.72 * ((0.434 * np.log(B)) ** 0.647)),
                   B)
    return fwi

def compute_fwi(dataset, ffmc_yesterday, dmc_yesterday, dc_yesterday):
    """
    Compute the full suite of FWI components given a dataset with the necessary variables.
    
    Parameters:
      dataset : xarray.Dataset
          Must contain variables: 't2m' (°C), 'RH' (%), 'windspeed' (km/h), and 'rain' (mm).
      ffmc_yesterday, dmc_yesterday, dc_yesterday:
          xarray.DataArray or np.array with yesterday's indices (initial conditions).
    
    Returns:
      updated_ds : xarray.Dataset
          Original dataset with added variables: 'FFMC', 'DMC', 'DC', 'ISI', 'BUI', and 'FWI'.
    """
    T = dataset['t2m_C']
    RH = dataset['RH']
    wind = dataset['windspeed']
    rain = dataset['tp_mm']
    
    FFMC = calc_ffmc(T, RH, wind, rain, ffmc_yesterday)
    DMC = calc_dmc(T, RH, rain, dmc_yesterday)
    DC  = calc_dc(T, rain, dc_yesterday)
    ISI = calc_isi(FFMC, wind)
    BUI = calc_bui(DMC, DC)
    FWI = calc_fwi(ISI, BUI)
    
    updated_ds = dataset.copy()
    updated_ds['FFMC'] = FFMC
    updated_ds['DMC'] = DMC
    updated_ds['DC']  = DC
    updated_ds['ISI'] = ISI
    updated_ds['BUI'] = BUI
    updated_ds['FWI'] = FWI
    return updated_ds




In [ ]:
### FWI Calculation #2
"""
Compute the Canadian Forest Fire Weather Index (FWI) from daily meteorological inputs.
The formulas are based on Van Wagner (1987) and subsequent refinements.
Note: This implementation assumes that:
    - Temperature is provided in °C (if using ERA5, convert from Kelvin by subtracting 273.15).
    - Relative Humidity is in %.
    - Wind speed is provided in km/h (if using m/s, multiply by 3.6).
    - Precipitation is in mm.
The system is recursive, so previous day values for FFMC, DMC, and DC must be supplied.
Also, following the ECMWF FWI implementation, if the temperature is below 6°C the FWI is set to 0.
"""

import numpy as np 
import xarray as xr 

def calc_ffmc(T, RH, wind, rain, ffmc_yesterday):
    """
    Calculate the Fine Fuel Moisture Code (FFMC).

    Parameters:
      T : xarray.DataArray or np.array
          Daily air temperature in °C.
      RH : xarray.DataArray or np.array
          Daily relative humidity in %.
      wind : xarray.DataArray or np.array
          Daily wind speed in km/h.
      rain : xarray.DataArray or np.array
          Daily precipitation in mm.
      ffmc_yesterday : xarray.DataArray or np.array
          Yesterday's FFMC (typically around 85).
          
    Returns:
      ffmc : xarray.DataArray or np.array
          Today's FFMC.
    """
    # Convert previous FFMC to moisture content (m_o)
    m_o = 147.2 * (101.0 - ffmc_yesterday) / (59.5 + ffmc_yesterday)
    
    # Rainfall effect: only effective if rain > 0.5 mm
    rf = xr.where(rain > 0.5, rain - 0.5, 0.0)
    
    # Update moisture content after rain (m_r)
    m_r = xr.where(
        rf > 0,
        xr.where(
            m_o <= 150.0,
            m_o + 42.5 * rf * np.exp(-100.0 / (251.0 - m_o)) * (1 - np.exp(-6.93 / rf)),
            m_o + 42.5 * rf * np.exp(-100.0 / (251.0 - m_o)) * (1 - np.exp(-6.93 / rf)) + 0.0015 * (m_o - 150.0)**2 * np.sqrt(rf)
        ),
        m_o
    )
    # Cap moisture content at 250
    m_r = xr.where(m_r > 250.0, 250.0, m_r)
    
    # Equilibrium moisture contents for drying (E_d) and wetting (E_w)
    E_d = 0.942 * (RH**0.679) + 11.0 * np.exp((RH - 100.0) / 10.0) + 0.18 * (21.1 - T) * (1 - np.exp(-0.115 * RH))
    E_w = 0.618 * (RH**0.753) + 10.0 * np.exp((RH - 100.0) / 10.0) + 0.18 * (21.1 - T) * (1 - np.exp(-0.115 * RH))
    
    # Drying or wetting phase:
    m = xr.where(
        m_r > E_d,
        E_d + (m_r - E_d) / (10 ** (
            (0.424 * (1 - (RH/100.0)**1.7)) +
            (0.0694 * np.sqrt(wind) * (1 - (RH/100.0)**8)) * (0.581 * np.exp(0.0365 * T))
        )),
        xr.where(
            m_r < E_w,
            E_w - (E_w - m_r) / (10 ** (
                (0.424 * (1 - ((100.0 - RH)/100.0)**1.7)) +
                (0.0694 * np.sqrt(wind) * (1 - ((100.0 - RH)/100.0)**8)) * (0.581 * np.exp(0.0365 * T))
            )),
            m_r
        )
    )
    
    # Convert moisture content back to FFMC
    ffmc = (59.5 * (250.0 - m)) / (147.2 + m)
    ffmc = xr.where(ffmc > 101.0, 101.0, ffmc)
    ffmc = xr.where(ffmc < 0.0, 0.0, ffmc)
    return ffmc

def calc_dmc(T, RH, rain, dmc_yesterday):
    """
    Calculate the Duff Moisture Code (DMC).
    """
    # Effective rainfall for DMC; threshold of 1.5 mm.
    rf = xr.where(rain > 1.5, rain, 0.0)
    Re = xr.where(rain > 1.5, 0.92 * rain - 1.27, 0.0)
    
    # Drying term based on T and RH:
    K = 1.894 * (T + 1.1) * (100 - RH) * 1e-4
    dmc = np.maximum(dmc_yesterday + Re + K, 0)
    return dmc

def calc_dc(T, rain, dc_yesterday):
    """
    Calculate the Drought Code (DC).
    """
    # Effective rainfall for DC: only if rain > 2.8 mm.
    rf = xr.where(rain > 2.8, rain, 0.0)
    Rd = xr.where(rain > 2.8, 0.83 * rain - 1.27, 0.0)
    V = xr.where(T >= -2.8, 0.36 * (T + 2.8), 0.0)
    dc = dc_yesterday + 0.5 * V
    Qo = 800 * np.exp(-dc_yesterday / 400.0)
    Qr = Qo + 3.937 * Rd
    Dr = 400 * np.log(800.0 / Qr)
    dc = xr.where(rain > 2.8, Dr, dc)
    dc = np.maximum(dc, 0)
    return dc

def calc_isi(ffmc, wind):
    """
    Calculate the Initial Spread Index (ISI) based on FFMC and wind.
    """
    # Convert FFMC to fine fuel moisture content (m)
    m = (59.5 * 250 - 147.2 * ffmc) / (ffmc + 59.5)
    f_F = 91.9 * np.exp(-0.1386 * m) * (1 + (m**5.31) / 4.93e7)
    f_W = np.exp(0.05039 * wind)
    isi = 0.208 * f_W * f_F
    return isi

def calc_bui(dmc, dc):
    """
    Calculate the Buildup Index (BUI) from DMC and DC.
    """
    bui = xr.zeros_like(dmc)
    cond = dmc <= 0.4 * dc
    bui = xr.where(cond, (0.8 * dmc * dc) / (dmc + 0.4 * dc),
                     dmc - (1 - 0.8 * dc / (dmc + 0.4 * dc)) * (0.92 + 0.0114 * dmc))
    bui = np.maximum(bui, 0)
    return bui

def calc_fwi(isi, bui):
    """
    Calculate the final Fire Weather Index (FWI) from ISI and BUI.
    """
    f_D = xr.where(bui <= 80, 0.626 * (bui ** 0.809) + 2,
                   1000.0 / (25.0 + 108.64 * np.exp(-0.023 * bui)))
    B = 0.1 * isi * f_D
    fwi = xr.where(B > 1, np.exp(2.72 * ((0.434 * np.log(B)) ** 0.647)),
                   B)
    return fwi

def compute_fwi(dataset, ffmc_yesterday, dmc_yesterday, dc_yesterday):
    """
    Compute the full suite of FWI components given a dataset with the necessary variables.
    ECMWF's approach sets FWI to 0 if conditions indicate minimal fire risk (e.g., winter).
    """
    T = dataset['t2m_C']      # Temperature in °C
    RH = dataset['RH']        # Relative humidity in %
    wind = dataset['windspeed']  # Wind speed in km/h
    rain = dataset['tp']      # Precipitation in mm
    
    # Compute the indices
    FFMC = calc_ffmc(T, RH, wind, rain, ffmc_yesterday)
    DMC = calc_dmc(T, RH, rain, dmc_yesterday)
    DC  = calc_dc(T, rain, dc_yesterday)
    ISI = calc_isi(FFMC, wind)
    BUI = calc_bui(DMC, DC)
    FWI = calc_fwi(ISI, BUI)
    
    # ECMWF FWI handling: if temperature is below 6°C, fire risk is negligible.
    FWI = xr.where(T < 6, 0.0, FWI)
    
    updated_ds = dataset.copy()
    updated_ds['FFMC'] = FFMC
    updated_ds['DMC'] = DMC
    updated_ds['DC']  = DC
    updated_ds['ISI'] = ISI
    updated_ds['BUI'] = BUI
    updated_ds['FWI'] = FWI
    return updated_ds


In [ ]:
ffmc0 = xr.full_like(era5_0['t2m_C'], 85.0)
dmc0  = xr.full_like(era5_0['t2m_C'], 6.0)
dc0   = xr.full_like(era5_0['t2m_C'], 15.0)
ds_out = compute_fwi(era5_0, ffmc0, dmc0, dc0)
ds_out

In [8]:
effis = gpd.read_file("data/processed/effis/effis_forests_EEA38.gpkg")
effis['geometry'] = effis['geometry'].centroid
effis = effis[effis['CountryAbbv'] != 'TR']

/tmp/ipykernel_41292/537074112.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  effis['geometry'] = effis['geometry'].centroid


In [ ]:
era5 = era5_0.copy()

In [9]:
# Ensure effis['Date'] is datetime, then to Period
effis['Date'] = pd.to_datetime(effis['Date'], errors='coerce')  # Handle invalid dates
effis['YearMonth'] = effis['Date'].dt.to_period('M')  # e.g., '2024-01'

# Ensure era5['valid_time'] is datetime, then to Period
# monthly_periods = era5['valid_time'].to_index().to_period('M')
# era5['valid_time'] = pd.to_datetime(era5['valid_time'], errors='coerce')  # Convert to datetime
# era5['valid_time'] = era5['valid_time'].to_pandas().dt.to_period('M')  # Convert to Period

In [10]:
effis['YearMonth'].isnull().unique()

array([False,  True])

In [11]:
# Remove the rows with null YearMonth
effis = effis.dropna(subset=['YearMonth'])

In [12]:
BioRegions = gpd.read_file('data/external/eea_v_3035_1_mio_biogeo-regions_p_2016_v01_r00/BiogeoRegions2016.shp')
# Ensure both datasets have the same CRS
if effis.crs != BioRegions.crs:
    BioRegions = BioRegions.to_crs(effis.crs)

# Perform spatial join
# 'within' ensures points are inside polygons; 'intersects' is an alternative
effis = gpd.sjoin(
    effis,
    BioRegions[['code', 'geometry']],  # Only keep 'code' and geometry from BioRegion
    how='left',                       # Keep all points, even if no match
    predicate='within'                # Points within polygons
)

# Rename the 'code' column to 'BioRegion'
effis = effis.rename(columns={'code': 'BioRegion'})

# Drop unnecessary columns from the join (e.g., index_right)
effis = effis.drop(columns=['index_right'], errors='ignore')

# Verify the result
print("Updated effis_clipped with BioRegion:")
print(effis.head())



Updated effis_clipped with BioRegion:
          Date CountryAbbv ForestType AREA_HA CountryName  \
311 2017-08-01          EL      Mixed      24        None   
312 2021-08-03          EL  Broadleaf     263        None   
313 2021-08-01          EL    Conifer      15        None   
314 2017-09-07          EL  Broadleaf      67        None   
315 2000-08-26          EL  Broadleaf    5245        None   

                            geometry_centroid                   geometry  \
311  (21.822589552506127, 36.945430416975384)  POINT (21.82259 36.94543)   
312   (21.78904842694764, 37.013927831018236)  POINT (21.78905 37.01393)   
313     (21.6322282643577, 38.41703143253353)  POINT (21.63223 38.41703)   
314  (20.637931528102943, 39.318146662437925)  POINT (20.63793 39.31815)   
315   (20.480534515228786, 39.77351374691639)  POINT (20.48053 39.77351)   

    YearMonth      BioRegion  
311   2017-08  Mediterranean  
312   2021-08  Mediterranean  
313   2021-08  Mediterranean  
314   2017-09 

In [ ]:
import geopandas as gpd
import xarray as xr
import pandas as pd
from cftime import DatetimeProlepticGregorian

# Assume effis is your GeoPandas DataFrame with polygons and 'YearMonth'
# Assume era5 is your xarray Dataset with 'FWI' and 'vpd'

# 1. Prepare effis
effis = effis.to_crs(epsg=4326)
effis['centroid'] = effis.geometry.centroid
effis['lon'] = effis['centroid'].x  # Longitude
effis['lat'] = effis['centroid'].y  # Latitude

# Subset for testing
effis_subset = effis.iloc[:10]
print(f"Number of points: {len(effis)}")
print(f"Sample coords: {effis[['lon', 'lat']].head().to_numpy().tolist()}")
print(f"Sample YearMonth: {effis['YearMonth'].head().tolist()}")

# 2. Inspect era5
print(f"era5 time: {era5['valid_time'].values[:5]}")
print(f"era5 lon range: {era5['x'].min().values}, {era5['x'].max().values}")
print(f"era5 lat range: {era5['y'].min().values}, {era5['y'].max().values}")

# 3. Convert YearMonth (Period) to cftime objects
def period_to_cftime(period):
    year = period.year
    month = period.month
    return DatetimeProlepticGregorian(year, month, 1)

effis['time'] = effis['YearMonth'].apply(period_to_cftime)

# 4. Extract data at points
results = []

for idx, row in effis.iterrows():
    # Select data at this point (only spatial, keep all times initially)
    point_data = era5[['FWI', 'vpd']].sel(
        x=row['lon'],
        y=row['lat'],
        method='nearest'
    )
    
    # Debug: Check point_data structure
    print(f"Point {idx}: point_data dims: {point_data.dims}")
    print(f"Point {idx}: point_data shape: {point_data.sizes}")
    
    # Convert to DataFrame
    df = point_data.to_dataframe().reset_index()
    
    # Filter by YearMonth
    df['year_month'] = df['valid_time'].apply(lambda t: f"{t.year}-{t.month:02d}")
    df = df[df['year_month'] == row['YearMonth'].strftime('%Y-%m')]
    df = df.drop(columns=['year_month'])  # Clean up
    
    df['point_idx'] = idx
    results.append(df)

# 5. Combine results
result = pd.concat(results, ignore_index=True)
print(f"Extracted data shape: {result.shape}")
print(result.head())

# 6. Merge with effis_subset
effis_df = effis.drop(columns=['geometry', 'centroid']).reset_index().rename(columns={'index': 'point_idx'})
result_with_effis = result.merge(effis_df, on='point_idx', how='left')

# 7. Output
print("Final result:")
print(result_with_effis.head())
result_with_effis.to_csv('data/processed/effis/extracted_era5_time_series.csv', index=False)

In [ ]:
era5_0

In [ ]:
import geopandas as gpd
import xarray as xr
import pandas as pd
from cftime import DatetimeProlepticGregorian

# Assume effis is your GeoPandas DataFrame with polygons and 'YearMonth'
# Assume era5 is your xarray Dataset with 'FWI' and 'vpd'

# 1. Prepare effis
effis = effis.to_crs(epsg=4326)
effis['centroid'] = effis.geometry.centroid
effis['lon'] = effis['centroid'].x  # Longitude
effis['lat'] = effis['centroid'].y  # Latitude

# Subset for testing (optional)
effis_subset = effis.iloc[:10]
print(f"Number of points: {len(effis)}")
print(f"Sample coords: {effis[['lon', 'lat']].head().to_numpy().tolist()}")

# 2. Inspect era5
print(f"era5 time range: {era5['valid_time'].values[0]} to {era5['valid_time'].values[-1]}")
print(f"era5 lon range: {era5['x'].min().values}, {era5['x'].max().values}")
print(f"era5 lat range: {era5['y'].min().values}, {era5['y'].max().values}")

# 3. Extract full time series at points
results = []

for idx, row in effis.iterrows():
    # Select data at this point (full time series)
    point_data = era5[['t2m_C','d2m_C','RH','tp_mm','vpd_ARM','vpd_tetens','windspeed_km_h']].sel(
        x=row['lon'],
        y=row['lat'],
        method='nearest'
    )
    
    # Debug: Check point_data structure
    print(f"Point {idx}: point_data dims: {point_data.dims}")
    print(f"Point {idx}: point_data shape: {point_data.sizes}")
    
    # Convert to DataFrame
    df = point_data.to_dataframe().reset_index()
    df['point_idx'] = idx
    results.append(df)

# 4. Combine results
result = pd.concat(results, ignore_index=True)
print(f"Extracted data shape: {result.shape}")
print(result.head())

# 5. Merge with effis
effis_df = effis.drop(columns=['geometry', 'centroid']).reset_index().rename(columns={'index': 'point_idx'})
result_with_effis = result.merge(effis_df, on='point_idx', how='left')

# 6. Output
print("Final result:")
print(result_with_effis.head())
result_with_effis.to_csv('data/processed/effis/extracted_era5_full_time_series.csv', index=False)

In [ ]:
# save result with effis to csv
result_with_effis.to_csv('data/processed/effis/extracted_era5_full_time_series.csv', index=False)

In [ ]:
result_with_effis['point_idx'].unique()